Run the scripts nmw_get_csvs.py and nmw_make_dfs.py before running this notebook.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

with open('data/states.pickle', 'rb') as fh:
    states_df = pickle.load(fh)
states_series = states_df['Abbreviation']
print(states_series.to_string())

In [ ]:
with open('data/nmw.pickle', 'rb') as fh:
    nmw_df = pickle.load(fh)
#nmw_df = nmw_df.set_index(['STATE', 'DATE'])
present_states = nmw_df.index.unique('STATE').values
absent_states = states_df[~states_df['Abbreviation'].isin(present_states)]
absent_states

In [28]:
nmw_by_year = nmw_df.unstack(level='STATE')
nmw_by_year.columns = nmw_by_year.columns.droplevel(0)
nmw_by_year

STATE,AK,AR,AZ,CA,CO,CT,DC,DE,FL,GA,...,RI,SD,TX,UT,VA,VT,WA,WI,WV,WY
DATE,,,,,,,,,,,,,,,,,,,,,
1968-01-01,2.10,NaN,NaN,1.65,1.25,1.40,1.40,1.25,NaN,NaN,...,1.40,NaN,NaN,1.15,NaN,1.40,1.60,1.25,1.00,1.20
1969-01-01,2.10,NaN,NaN,1.65,1.25,1.40,1.40,1.25,NaN,NaN,...,1.40,NaN,NaN,1.15,NaN,1.40,1.60,1.25,1.00,1.20
1970-01-01,2.10,1.10,NaN,1.65,1.25,1.60,2.00,1.25,NaN,NaN,...,1.60,1.00,NaN,1.15,NaN,1.60,1.60,1.30,1.00,1.30
1971-01-01,2.10,1.10,NaN,1.65,1.25,1.60,2.00,1.25,NaN,NaN,...,1.60,1.00,NaN,1.15,NaN,1.60,1.60,1.30,1.00,1.30
1972-01-01,2.10,1.20,NaN,1.65,1.25,1.85,2.25,1.60,NaN,1.25,...,1.60,1.00,1.40,1.35,NaN,1.60,1.60,1.45,1.20,1.50
1973-01-01,2.10,1.20,NaN,1.65,1.25,1.85,2.25,1.60,NaN,1.25,...,1.60,1.00,1.40,1.35,NaN,1.60,1.60,1.45,1.20,1.50
1974-01-01,2.10,1.20,NaN,1.65,1.25,1.85,2.25,1.60,NaN,1.25,...,1.60,1.00,1.40,1.35,NaN,1.60,1.60,1.45,1.20,1.50
1975-01-01,2.10,1.20,NaN,1.65,1.25,1.85,2.25,1.60,NaN,1.25,...,1.60,1.00,1.40,1.35,NaN,1.60,1.60,1.45,1.20,1.50
1976-01-01,2.80,1.90,NaN,2.00,1.25,2.31,2.75,2.00,NaN,1.25,...,2.30,2.00,1.40,1.70,2.00,2.30,2.30,2.10,2.00,1.60


In [ ]:
print(nmw_by_year.columns)
up_2024 = nmw_by_year.diff().loc['2024', :]
print(up_2024.columns)

In [ ]:
nmw_gb = nmw_df.groupby(level=0, as_index=False)
earliest_reports = nmw_gb.nth(0).sort_index(level='DATE', ascending=False)
earliest_reports

In [ ]:
nmw2324_df = nmw_gb.nth[-2:].unstack()
nmw2324_df.columns = [str(x[1].year) for x in nmw2324_df.columns]
nmw2324_df = nmw2324_df.loc[:, ['2023','2024']].sort_values('STATE').dropna(how='any')
nmw2324_df['DIFF'] = nmw2324_df['2024'] - nmw2324_df['2023']
nmw2324_df = nmw2324_df[nmw2324_df['DIFF'] > 0]
nmw2324_df.sort_values(by='DIFF', ascending=False).head()

In [ ]:
nmw2324_df['% DIFF'] = 100 * nmw2324_df['DIFF'] / nmw2324_df['2023']
nmw2324_df.sort_values(by='% DIFF', ascending=False).head()

In [ ]:
nmw_dfr = nmw_df.reset_index()
nmw_dfr['YEAR'] = nmw_dfr['DATE'].dt.year
nmw_dfr['DECADE'] = (nmw_dfr['YEAR'] - nmw_dfr['YEAR'].mod(10)).mod(100)
decade_av = nmw_dfr.groupby(by='DECADE')['MIN_WAGE'].mean()
xs = [60, 70, 80, 90, 0, 10, 20]
decade_av = decade_av.reindex(xs)
print(decade_av.to_string())

In [ ]:

xlabels = [f'{x:02d}s' for x in xs]
ys = decade_av.values
plt.plot(xlabels, ys)
plt.xlabel('Decade')
plt.ylabel('Avg. min. wage')
plt.show()